In [ ]:
pip install numpy pandas matplotlib scikit-learn


Note: you may need to restart the kernel to use updated packages.


In [ ]:
# import pandas as pd
# import numpy as np
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import PolynomialFeatures, StandardScaler
# from sklearn.metrics import mean_squared_error

# # -------------------- Load & Clean --------------------
# df = pd.read_csv("processed_features.csv")
# df = df.fillna(df.mean(numeric_only=True))  # column-mean imputation

# X = df.drop(["Mu"], axis=1).to_numpy(dtype=np.float64)
# y = df["Mu"].to_numpy(dtype=np.float64)
# y = np.nan_to_num(y, nan=np.nanmean(y)).astype(np.float64)

# # -------------------- Split then scale base X --------------------
# X_train, X_test, y_train, y_test = train_test_split(
#     X, y, test_size=0.2, random_state=42
# )
# base_scaler = StandardScaler()
# X_train = base_scaler.fit_transform(X_train).astype(np.float64)
# X_test  = base_scaler.transform(X_test).astype(np.float64)

# # -------------------- Safe GD helper --------------------
# def gd_fit(
#     Xm, ym, lr=1e-3, iters=8000, l2=1e-4, clip=1.0
# ):
#     """
#     Batch gradient descent minimizing (1/m)||Xθ - y||^2 + l2||θ_{no-bias}||^2.
#     - l2 not applied to bias term.
#     - gradient clipping by global norm.
#     """
#     m, d = Xm.shape
#     theta = np.zeros(d, dtype=np.float64)
#     for _ in range(iters):
#         err = Xm @ theta - ym  # shape (m,)
#         if not np.isfinite(err).all():
#             return None  # signal divergence

#         # grad of MSE part: (2/m) X^T (Xθ - y)
#         grad = (2.0 / m) * (Xm.T @ err)

#         # L2 (exclude bias at index 0)
#         grad[1:] += 2.0 * l2 * theta[1:]

#         # gradient clipping
#         gnorm = np.linalg.norm(grad)
#         if not np.isfinite(gnorm):
#             return None
#         if gnorm > clip:
#             grad = grad * (clip / gnorm)

#         theta -= lr * grad

#         if not np.isfinite(theta).all():
#             return None
#     return theta

# def standardize_poly_keep_bias(X_train_poly, X_test_poly):
#     """
#     Standardize polynomial features EXCEPT the bias column (col 0).
#     Returns standardized versions with the bias column intact.
#     """
#     Xtr = X_train_poly.copy().astype(np.float64)
#     Xte = X_test_poly.copy().astype(np.float64)

#     if Xtr.shape[1] > 1:
#         scaler = StandardScaler()
#         Xtr[:, 1:] = scaler.fit_transform(Xtr[:, 1:])
#         Xte[:, 1:] = scaler.transform(Xte[:, 1:])

#     # Ensure exact bias column of ones
#     Xtr[:, 0] = 1.0
#     Xte[:, 0] = 1.0
#     return Xtr, Xte

# # -------------------- Train/eval degrees 0..7 with safeguards --------------------
# degree_rmse = []
# print("\n---- Degree (0–7) vs RMSE (Stable Gradient Descent) ----")

# lrs_to_try = [1e-2, 5e-3, 1e-3, 5e-4, 1e-4]

# for deg in range(0, 8):
#     poly = PolynomialFeatures(degree=deg, include_bias=True)
#     X_train_poly = poly.fit_transform(X_train)
#     X_test_poly  = poly.transform(X_test)

#     # Re-standardize expanded features (except bias) to avoid huge magnitudes
#     X_train_poly, X_test_poly = standardize_poly_keep_bias(X_train_poly, X_test_poly)

#     theta = None
#     for lr in lrs_to_try:
#         theta = gd_fit(
#             X_train_poly, y_train,
#             lr=lr, iters=10000 if deg >= 4 else 8000,
#             l2=1e-4 if deg <= 3 else 5e-4,   # a bit more L2 for higher degrees
#             clip=1.0
#         )
#         if theta is not None:
#             break  # succeeded at this lr

#     if theta is None:
#         print(f"Degree {deg:2d} | RMSE = NaN (diverged even with small LR)")
#         degree_rmse.append([deg, np.nan])
#         continue

#     y_pred = X_test_poly @ theta
#     y_pred = np.nan_to_num(y_pred, posinf=1e308, neginf=-1e308)

#     rmse = np.sqrt(mean_squared_error(y_test, y_pred))
#     degree_rmse.append([deg, rmse])
#     print(f"Degree {deg:2d} | RMSE = {rmse:.6f}")

# # -------------------- Save RMSE table --------------------
# rmse_df = pd.DataFrame(degree_rmse, columns=["Degree", "RMSE"])
# rmse_df.to_csv("degree_rmse_gd.csv", index=False)
# print("\n📁 Saved: degree_rmse_gd.csv")


In [ ]:
# poly_gd_and_ridge_contours.py
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from dataclasses import dataclass
from typing import Tuple, List, Optional
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.metrics import mean_squared_error

# ========= CONFIG =========
CSV_PATH = "processed_features.csv"   # <- change if needed
TARGET   = "Mu"
DEG_RANGE = range(0,6)               # degrees 0..5  (edit if you want 0..11)
TEST_SIZE = 0.2
RANDOM_STATE = 42
# learning rates to try for stability at each degree
LR_GRID = [1e-2, 5e-3, 1e-3, 5e-4, 1e-4]
# Ridge alpha search space (log grid)
ALPHAS = np.logspace(-5, 1, 13)       # 1e-5 ... 10
MAX_ITERS = 8000
MAX_ITERS_HIGH_DEG = 10000
GRAD_CLIP = 1.0

# ========= UTILITIES =========
def standardize_poly_keep_bias(Xtr, Xte):
    """Standardize all columns except the first (bias). Ensures col0==1."""
    Xtr = Xtr.astype(np.float64).copy()
    Xte = Xte.astype(np.float64).copy()
    if Xtr.shape[1] > 1:
        scaler = StandardScaler()
        Xtr[:, 1:] = scaler.fit_transform(Xtr[:, 1:])
        Xte[:, 1:] = scaler.transform(Xte[:, 1:])
    Xtr[:, 0] = 1.0
    Xte[:, 0] = 1.0
    return Xtr, Xte

@dataclass
class GDResult:
    theta: np.ndarray
    theta_hist: np.ndarray
    rmse_hist_train: List[float]

def gd_fit(
    X: np.ndarray,
    y: np.ndarray,
    lr: float = 1e-3,
    iters: int = 8000,
    l2: float = 0.0,
    clip: float = 1.0,
) -> Optional[GDResult]:
    """
    Batch GD minimizing MSE + l2*||theta_{1:}||^2. No regularization on bias.
    Keeps parameter trajectory and train RMSE per iter.
    Returns None if numerics blow up.
    """
    m, d = X.shape
    theta = np.zeros(d, dtype=np.float64)
    theta_hist = np.empty((iters + 1, d), dtype=np.float64)
    theta_hist[0] = theta
    rmse_hist = []

    for t in range(1, iters + 1):
        err = X @ theta - y
        if not np.isfinite(err).all():
            return None
        grad = (2.0 / m) * (X.T @ err)
        grad[1:] += 2.0 * l2 * theta[1:]

        # clip
        gnorm = np.linalg.norm(grad)
        if not np.isfinite(gnorm):
            return None
        if gnorm > clip:
            grad = grad * (clip / gnorm)

        theta -= lr * grad
        if not np.isfinite(theta).all():
            return None

        theta_hist[t] = theta
        rmse_hist.append(np.sqrt(mean_squared_error(y, X @ theta)))

    return GDResult(theta=theta, theta_hist=theta_hist, rmse_hist_train=rmse_hist)

def grid_train_gd(
    Xtr, ytr, Xte, yte, deg: int, l2: float
) -> Tuple[Optional[GDResult], float, float, float]:
    iters = MAX_ITERS_HIGH_DEG if deg >= 4 else MAX_ITERS
    for lr in LR_GRID:
        res = gd_fit(Xtr, ytr, lr=lr, iters=iters, l2=l2, clip=GRAD_CLIP)
        if res is None:
            continue
        ytr_hat = Xtr @ res.theta
        yte_hat = Xte @ res.theta
        tr_rmse = np.sqrt(mean_squared_error(ytr, ytr_hat))
        te_rmse = np.sqrt(mean_squared_error(yte, yte_hat))
        return res, tr_rmse, te_rmse, lr
    return None, np.nan, np.nan, np.nan

def make_contour(
    X: np.ndarray,
    y: np.ndarray,
    theta_ref: np.ndarray,
    l2: float,
    title: str,
    out_path: str,
    path_thetas: Optional[np.ndarray] = None,
    idx0: int = 0,
    idx1: int = 1,
    grid_span: float = 2.5,
    grid_pts: int = 120,
):
    """
    2D contour of objective in the plane of (theta[idx0], theta[idx1]),
    keeping other params fixed to theta_ref.
    """
    t0_c, t1_c = theta_ref[idx0], theta_ref[idx1]
    t0s = np.linspace(t0_c - grid_span, t0_c + grid_span, grid_pts)
    t1s = np.linspace(t1_c - grid_span, t1_c + grid_span, grid_pts)
    Z = np.zeros((grid_pts, grid_pts), dtype=np.float64)

    base = theta_ref.copy()
    for i, v0 in enumerate(t0s):
        for j, v1 in enumerate(t1s):
            theta = base.copy()
            theta[idx0] = v0
            theta[idx1] = v1
            resid = X @ theta - y
            mse = np.mean(resid**2)
            # ridge penalty excludes bias index 0
            penalty = l2 * np.sum(theta[1:] ** 2)
            Z[j, i] = mse + penalty  # plot objective (not RMSE) for smoother contours

    plt.figure(figsize=(8, 6))
    CS = plt.contour(t0s, t1s, Z, levels=30)
    plt.clabel(CS, inline=True, fontsize=8)
    if path_thetas is not None:
        plt.plot(path_thetas[:, idx0], path_thetas[:, idx1], marker=".", linewidth=1.5, label="GD path")
        plt.legend()
    plt.xlabel(f"Theta[{idx0}]")
    plt.ylabel(f"Theta[{idx1}]")
    plt.title(title)
    plt.tight_layout()
    plt.savefig(out_path, dpi=160)
    plt.close()

# ========= MAIN WORKFLOW =========
def main():
    # --- load & clean ---
    if not os.path.exists(CSV_PATH):
        raise FileNotFoundError(f"Could not find {CSV_PATH}. Put your CSV next to this script or update CSV_PATH.")
    df = pd.read_csv(CSV_PATH)
    df = df.fillna(df.mean(numeric_only=True))

    X = df.drop([TARGET], axis=1).to_numpy(dtype=np.float64)
    y = df[TARGET].to_numpy(dtype=np.float64)
    y = np.nan_to_num(y, nan=np.nanmean(y)).astype(np.float64)

    # --- split & scale base features ---
    Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE)
    scaler = StandardScaler()
    Xtr = scaler.fit_transform(Xtr).astype(np.float64)
    Xte  = scaler.transform(Xte).astype(np.float64)

    # ====== PART A: Plain GD (MSE) — degree selection ======
    rows = []
    best_deg = None
    best_rmse = np.inf
    best_pack = None  # (deg, res, Xtrp, Xtep, poly)

    print("\n---- Degree vs RMSE (Gradient Descent) ----")
    for deg in DEG_RANGE:
        poly = PolynomialFeatures(degree=deg, include_bias=True)
        Xtrp = poly.fit_transform(Xtr)
        Xtep = poly.transform(Xte)
        Xtrp, Xtep = standardize_poly_keep_bias(Xtrp, Xtep)

        # a tiny bit more L2 for higher degrees to keep things tame
        l2 = 1e-4 if deg <= 3 else 5e-4
        res, tr_rmse, te_rmse, lr = grid_train_gd(Xtrp, ytr, Xtep, yte, deg, l2=l2)
        if res is None:
            print(f"Degree {deg:2d} | RMSE_test = NaN (diverged)")
            rows.append([deg, np.nan, np.nan, np.nan, np.nan])
            continue

        print(f"Degree {deg:2d} | RMSE_train = {tr_rmse:.6f} | RMSE_test = {te_rmse:.6f} | lr={lr:g}")
        rows.append([deg, tr_rmse, te_rmse, lr, l2])

        if te_rmse < best_rmse:
            best_rmse = te_rmse
            best_deg = deg
            best_pack = (deg, res, Xtrp, Xtep, poly, l2)

    rmse_df = pd.DataFrame(rows, columns=["Degree", "Train_RMSE", "Test_RMSE", "LR_used", "L2_used"])
    rmse_df.to_csv("gd_degree_rmse.csv", index=False)
    print("Saved: gd_degree_rmse.csv")

    # --- Detailed outputs for the best degree (plain GD) ---
    deg, res, Xtrp, Xtep, poly, l2 = best_pack
    print(f"\nBest (plain GD) degree = {deg} with Test RMSE = {best_rmse:.6f}")

    # print RMSE every 50 iterations
    print("\nGD Train RMSE every 50 iterations (best degree):")
    for i in range(49, len(res.rmse_hist_train), 50):
        print(f"iter {i+1:4d}: {res.rmse_hist_train[i]:.6f}")

    # contour with GD path (use first two parameters)
    make_contour(
        Xtrp, ytr, res.theta, l2=l2,
        title=f"GD Objective Contour (degree={deg}) with path",
        out_path=f"gd_contour_degree_{deg}.png",
        path_thetas=res.theta_hist
    )
    print(f"Saved: gd_contour_degree_{deg}.png")

    # ====== PART B: Ridge (GD with L2) — degree & alpha selection ======
    ridge_rows = []
    ridge_best = (np.inf, None)  # (test_rmse, pack)
    print("\n---- Degree/Alpha vs RMSE (Ridge via GD) ----")
    for deg in DEG_RANGE:
        poly = PolynomialFeatures(degree=deg, include_bias=True)
        Xtrp = poly.fit_transform(Xtr)
        Xtep = poly.transform(Xte)
        Xtrp, Xtep = standardize_poly_keep_bias(Xtrp, Xtep)

        best_alpha_for_deg = None
        best_te_rmse_for_deg = np.inf
        best_res_for_deg = None

        for alpha in ALPHAS:
            res, tr_rmse, te_rmse, lr = grid_train_gd(Xtrp, ytr, Xtep, yte, deg, l2=alpha)
            if res is None:
                continue
            ridge_rows.append([deg, float(alpha), tr_rmse, te_rmse, lr])
            if te_rmse < best_te_rmse_for_deg:
                best_te_rmse_for_deg = te_rmse
                best_alpha_for_deg = float(alpha)
                best_res_for_deg = (res, lr)

        if best_alpha_for_deg is not None:
            print(f"Degree {deg:2d} | best alpha={best_alpha_for_deg:g} | RMSE_test={best_te_rmse_for_deg:.6f}")
            if best_te_rmse_for_deg < ridge_best[0]:
                ridge_best = (
                    best_te_rmse_for_deg,
                    (deg, best_alpha_for_deg, best_res_for_deg[0], Xtrp, Xtep, poly)
                )
        else:
            print(f"Degree {deg:2d} | all alphas diverged")

    ridge_df = pd.DataFrame(ridge_rows, columns=["Degree", "Alpha", "Train_RMSE", "Test_RMSE", "LR_used"])
    ridge_df.to_csv("ridge_degree_alpha_rmse.csv", index=False)
    print("Saved: ridge_degree_alpha_rmse.csv")

    # --- Detailed outputs for Ridge best ---
    ridge_te_rmse, pack = ridge_best
    rdeg, ralpha, rres, RXtrp, RXtep, Rpoly = pack
    print(f"\nBest Ridge: degree={rdeg}, alpha={ralpha:g}, Test RMSE={ridge_te_rmse:.6f}")

    # print RMSE every 50 iterations (ridge)
    print("\nRidge-GD Train RMSE every 50 iterations (best degree/alpha):")
    for i in range(49, len(rres.rmse_hist_train), 50):
        print(f"iter {i+1:4d}: {rres.rmse_hist_train[i]:.6f}")

    # contour with GD path for ridge objective
    make_contour(
        RXtrp, ytr, rres.theta, l2=ralpha,
        title=f"Ridge Objective Contour (degree={rdeg}, alpha={ralpha:g}) with path",
        out_path=f"ridge_contour_degree_{rdeg}_alpha_{ralpha:g}.png",
        path_thetas=rres.theta_hist
    )
    print(f"Saved: ridge_contour_degree_{rdeg}_alpha_{ralpha:g}.png")

if __name__ == "__main__":
    main()



---- Degree vs RMSE (Gradient Descent) ----
Degree  0 | RMSE_train = 22.015441 | RMSE_test = 15.575324 | lr=0.01
Degree  1 | RMSE_train = 20.096732 | RMSE_test = 14.864647 | lr=0.01
Degree  2 | RMSE_train = 14.277297 | RMSE_test = 22.699547 | lr=0.01
Degree  3 | RMSE_train = 6.991245 | RMSE_test = 51.622014 | lr=0.01
Degree  4 | RMSE_train = 5.358354 | RMSE_test = 46.740635 | lr=0.01
Degree  5 | RMSE_train = 6.865224 | RMSE_test = 34.911918 | lr=0.01
Saved: gd_degree_rmse.csv

Best (plain GD) degree = 1 with Test RMSE = 14.864647

GD Train RMSE every 50 iterations (best degree):
iter   50: 22.685330
iter  100: 22.408780
iter  150: 22.153319
iter  200: 21.917958
iter  250: 21.701600
iter  300: 21.503096
iter  350: 21.321305
iter  400: 21.155153
iter  450: 21.003676
iter  500: 20.866043
iter  550: 20.741558
iter  600: 20.629644
iter  650: 20.529816
iter  700: 20.441653
iter  750: 20.364767
iter  800: 20.298775
iter  850: 20.243276
iter  900: 20.197830
iter  950: 20.161927
iter 1000: 20.